In [5]:
import numpy as np
import pandas as pd
from scipy import sparse
import xgboost as xgb
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import train_test_split
import math
import graphlab
from sklearn.cluster import DBSCAN

print('beginning processing....')

train_abc = pd.read_json("~/Downloads/train.json")[['street_address','display_address']]
test_abc  = pd.read_json("~/Downloads/test.json")[['street_address','display_address']]
tt_abc = train_abc.append(test_abc)
levestein_distance = []
cosine_distance  = []
street_address = np.array(tt_abc['street_address'])
display_address = np.array(tt_abc['display_address'])
for i in xrange(len(tt_abc)):
    levestein_distance.append(graphlab.distances.levenshtein(street_address[i].lower(),display_address[i].lower()))


train_df = pd.read_json("~/Downloads/train.json")
test_df = pd.read_json("~/Downloads/test.json")
image_date = pd.read_csv("~/Downloads/listing_image_time.csv")


def runXGB(train_X, train_y, test_X,test_y,test_final,feature_names=None, seed_val=321, num_rounds=2000):
    param = {}
    param['objective'] = 'multi:softprob'
    param['eta'] = 0.02
    param['max_depth'] = 6
    param['silent'] = 0
    param['num_class'] = 3
    param['eval_metric'] = "mlogloss"
    param['min_child_weight'] = 5
    #param['subsample'] = 0.7
    #param['colsample_bytree'] = 0.7
    param['lambda'] = 2
    param['alpha'] = 0.02
    param['subsample'] = 0.8
    param['colsample_bytree'] = 0.8
    param['seed'] = seed_val
    num_rounds = num_rounds

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist,verbose_eval=10,early_stopping_rounds=20)
    
    xgtest1 = xgb.DMatrix(test_final)
    pred_test_y = model.predict(xgtest1)
    return pred_test_y, model

test_df["bathrooms"].loc[19671] = 1.5
test_df["bathrooms"].loc[22977] = 2.0
test_df["bathrooms"].loc[63719] = 2.0
train_df["price"] = train_df["price"].clip(upper=13000)

train_df["logprice"] = np.log(train_df["price"])
test_df["logprice"] = np.log(test_df["price"])

train_df["price_t"] =train_df["price"]/train_df["bedrooms"]
test_df["price_t"] = test_df["price"]/test_df["bedrooms"] 

train_df["room_sum"] = train_df["bedrooms"]+train_df["bathrooms"] 
test_df["room_sum"] = test_df["bedrooms"]+test_df["bathrooms"] 

train_df['price_per_room'] = train_df['price']/train_df['room_sum']
test_df['price_per_room'] = test_df['price']/test_df['room_sum']

train_df["num_photos"] = train_df["photos"].apply(len)
test_df["num_photos"] = test_df["photos"].apply(len)

train_df["num_features"] = train_df["features"].apply(len)
test_df["num_features"] = test_df["features"].apply(len)

train_df["num_description_words"] = train_df["description"].apply(lambda x: len(x.split(" ")))
test_df["num_description_words"] = test_df["description"].apply(lambda x: len(x.split(" ")))

train_df["created"] = pd.to_datetime(train_df["created"])
test_df["created"] = pd.to_datetime(test_df["created"])
train_df["created_year"] = train_df["created"].dt.year
test_df["created_year"] = test_df["created"].dt.year
train_df["created_month"] = train_df["created"].dt.month
test_df["created_month"] = test_df["created"].dt.month
train_df["created_day"] = train_df["created"].dt.day
test_df["created_day"] = test_df["created"].dt.day
train_df["created_hour"] = train_df["created"].dt.hour
test_df["created_hour"] = test_df["created"].dt.hour

train_df["pos"] = train_df.longitude.round(3).astype(str) + '_' + train_df.latitude.round(3).astype(str)
test_df["pos"] = test_df.longitude.round(3).astype(str) + '_' + test_df.latitude.round(3).astype(str)

image_date.columns = ["listing_id", "time_stamp"]
image_date.loc[80240,"time_stamp"] = 1478129766 

image_date["img_date"]                  = pd.to_datetime(image_date["time_stamp"], unit="s")
image_date["img_days_passed"]           = (image_date["img_date"].max() - image_date["img_date"]).astype("timedelta64[D]").astype(int)
image_date["img_date_month"]            = image_date["img_date"].dt.month
image_date["img_date_week"]             = image_date["img_date"].dt.week
image_date["img_date_day"]              = image_date["img_date"].dt.day
image_date["img_date_dayofweek"]        = image_date["img_date"].dt.dayofweek
image_date["img_date_dayofyear"]        = image_date["img_date"].dt.dayofyear
image_date["img_date_hour"]             = image_date["img_date"].dt.hour
image_date["img_date_monthBeginMidEnd"] = image_date["img_date_day"].apply(lambda x: 1 if x<10 else 2 if x<20 else 3)

train_df = pd.merge(train_df, image_date, on="listing_id", how="left")
test_df = pd.merge(test_df, image_date, on="listing_id", how="left")
print('Checkpoint-1')

import math
def cart2rho(x, y):
    rho = np.sqrt(x**2 + y**2)
    return rho


def cart2phi(x, y):
    phi = np.arctan2(y, x)
    return phi


def rotation_x(row, alpha):
    x = row['latitude']
    y = row['longitude']
    return x*math.cos(alpha) + y*math.sin(alpha)


def rotation_y(row, alpha):
    x = row['latitude']
    y = row['longitude']
    return y*math.cos(alpha) - x*math.sin(alpha)


def add_rotation(degrees, df):
    namex = "rot" + str(degrees) + "_X"
    namey = "rot" + str(degrees) + "_Y"

    df['num_' + namex] = df.apply(lambda row: rotation_x(row, math.pi/(180/degrees)), axis=1)
    df['num_' + namey] = df.apply(lambda row: rotation_y(row, math.pi/(180/degrees)), axis=1)

    return df

def operate_on_coordinates(tr_df, te_df):
    for df in [tr_df, te_df]:
        #polar coordinates system
        df["num_rho"] = df.apply(lambda x: cart2rho(x["latitude"] - 40.78222222, x["longitude"]+73.96527777), axis=1)
        df["num_phi"] = df.apply(lambda x: cart2phi(x["latitude"] - 40.78222222, x["longitude"]+73.96527777), axis=1)
        #rotations
        for angle in [15,30,45,60]:
            df = add_rotation(angle, df)

    return tr_df, te_df

train_df, test_df = operate_on_coordinates(train_df, test_df)

import re

def cap_share(x):
    return sum(1 for c in x if c.isupper())/float(len(x)+1)

for df in [train_df, test_df]:
    # do you think that users might feel annoyed BY A DESCRIPTION THAT IS SHOUTING AT THEM?
    df['num_cap_share'] = df['description'].apply(cap_share)
    
    # how long in lines the desc is?
    df['num_nr_of_lines'] = df['description'].apply(lambda x: x.count('<br /><br />'))
   
    # is the description redacted by the website?        
    df['num_redacted'] = 0
    df['num_redacted'].ix[df['description'].str.contains('website_redacted')] = 1

    
    # can we contact someone via e-mail to ask for the details?
    df['num_email'] = 0
    df['num_email'].ix[df['description'].str.contains('@')] = 1
    
    #and... can we call them?
    
    reg = re.compile(".*?(\(?\d{3}\D{0,3}\d{3}\D{0,3}\d{4}).*?", re.S)
    def try_and_find_nr(description):
        if reg.match(description) is None:
            return 0
        return 1

    df['num_phone_nr'] = df['description'].apply(try_and_find_nr)



## latitude longitude cluster code begins 
def get_centermost_point(cluster):
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
    centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
    return tuple(centermost_point)

print(len(train_df))
print(len(test_df))
coords = train_df.as_matrix(columns=['latitude', 'longitude'])
coords1 = test_df.as_matrix(columns=['latitude', 'longitude'])
kms_per_radian = 6371.0088
epsilon = 1.5 / kms_per_radian
db = DBSCAN(eps=epsilon, min_samples=1, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
cluster_labels = db.labels_
num_clusters = len(set(cluster_labels))
#clusters = pd.Series([coords[cluster_labels == n] for n in range(num_clusters)])
#print('Number of clusters: {}'.format(num_clusters))
#centermost_points = clusters.map(get_centermost_point)
#lats, lons = zip(*centermost_points)
#rep_points = pd.DataFrame({'lon':lons, 'lat':lats})
print('Checkpoint-2')
#df['cluster'] = rep_points.apply(lambda row: df[(train_df['latitude']==row['lat']) &amp;&amp; (df['lon']==row['lon'])].iloc[0], axis=1)
train_df['cluster_labels'] = cluster_labels
test_df['cluster_labels'] = db.fit_predict(np.radians(coords1))
df = train_df.append(test_df)
df['levestein_dist'] = levestein_distance
train_df = df[0:49352]
test_df = df[49352:]

##  latitude longitude cluster code begins 

vals = train_df['pos'].value_counts()
dvals = vals.to_dict()
train_df["density"] = train_df['pos'].apply(lambda x: dvals.get(x, vals.min()))
test_df["density"] = test_df['pos'].apply(lambda x: dvals.get(x, vals.min()))

features_to_use=["bathrooms", "bedrooms", "latitude", "longitude", "price","price_t","price_per_room", "logprice", "density",
"num_photos", "num_features", "num_description_words","listing_id", "created_year", "created_month", "created_day", "created_hour","img_days_passed","img_date_month","img_date_week","img_date_day","img_date_dayofweek","img_date_dayofyear","img_date_hour","img_date_monthBeginMidEnd","cluster_labels","levestein_dist"]
print('Checkpoint-3')

index=list(range(train_df.shape[0]))
random.shuffle(index)
a=[np.nan]*len(train_df)
b=[np.nan]*len(train_df)
c=[np.nan]*len(train_df)

for i in range(5):
    building_level={}
    for j in train_df['manager_id'].values:
        building_level[j]=[0,0,0]
    
    test_index=index[int((i*train_df.shape[0])/5):int(((i+1)*train_df.shape[0])/5)]
    train_index=list(set(index).difference(test_index))
    
    for j in train_index:
        temp=train_df.iloc[j]
        if temp['interest_level']=='low':
            building_level[temp['manager_id']][0]+=1
        if temp['interest_level']=='medium':
            building_level[temp['manager_id']][1]+=1
        if temp['interest_level']=='high':
            building_level[temp['manager_id']][2]+=1
            
    for j in test_index:
        temp=train_df.iloc[j]
        if sum(building_level[temp['manager_id']])!=0:
            a[j]=building_level[temp['manager_id']][0]*1.0/sum(building_level[temp['manager_id']])
            b[j]=building_level[temp['manager_id']][1]*1.0/sum(building_level[temp['manager_id']])
            c[j]=building_level[temp['manager_id']][2]*1.0/sum(building_level[temp['manager_id']])
            
train_df['manager_level_low']=a
train_df['manager_level_medium']=b
train_df['manager_level_high']=c

a=[]
b=[]
c=[]
building_level={}
for j in train_df['manager_id'].values:
    building_level[j]=[0,0,0]

for j in range(train_df.shape[0]):
    temp=train_df.iloc[j]
    if temp['interest_level']=='low':
        building_level[temp['manager_id']][0]+=1
    if temp['interest_level']=='medium':
        building_level[temp['manager_id']][1]+=1
    if temp['interest_level']=='high':
        building_level[temp['manager_id']][2]+=1

for i in test_df['manager_id'].values:
    if i not in building_level.keys():
        a.append(np.nan)
        b.append(np.nan)
        c.append(np.nan)
    else:
        a.append(building_level[i][0]*1.0/sum(building_level[i]))
        b.append(building_level[i][1]*1.0/sum(building_level[i]))
        c.append(building_level[i][2]*1.0/sum(building_level[i]))
test_df['manager_level_low']=a
test_df['manager_level_medium']=b
test_df['manager_level_high']=c

features_to_use.append('manager_level_low') 
features_to_use.append('manager_level_medium') 
features_to_use.append('manager_level_high')

categorical = ["display_address", "manager_id", "building_id","cluster_labels"]
for f in categorical:
        if train_df[f].dtype=='object':
            lbl = LabelEncoder()
            lbl.fit(list(train_df[f].values) + list(test_df[f].values))
            train_df[f] = lbl.transform(list(train_df[f].values))
            test_df[f] = lbl.transform(list(test_df[f].values))
            features_to_use.append(f)

train_df['features'] = train_df["features"].apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x]))
test_df['features'] = test_df["features"].apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x]))

tfidf = CountVectorizer(stop_words='english', max_features=200)
tr_sparse = tfidf.fit_transform(train_df["features"])
te_sparse = tfidf.transform(test_df["features"])

train_X = sparse.hstack([train_df[features_to_use], tr_sparse]).tocsr()
test_X = sparse.hstack([test_df[features_to_use], te_sparse]).tocsr()

target_num_map = {'high':0, 'medium':1, 'low':2}
train_y = np.array(train_df['interest_level'].apply(lambda x: target_num_map[x]))
print('Checkpoint-4')
X_train, X_test, y_train, y_test = train_test_split(train_X,train_y, test_size=0.2, random_state=0)
preds, model = runXGB(X_train,y_train,X_test,y_test,test_X, num_rounds=2000)
out_df = pd.DataFrame(preds)
out_df.columns = ["high", "medium", "low"]
out_df["listing_id"] = test_df.listing_id.values
out_df.to_csv("sub54_new.csv", index=False)

beginning processing....
Checkpoint-1
49352
74659
Checkpoint-2


/home/santanu/tensorflowGPU/lib/python2.7/site-packages/ipykernel/__main__.py:230: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/santanu/tensorflowGPU/lib/python2.7/site-packages/ipykernel/__main__.py:231: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Checkpoint-3


/home/santanu/tensorflowGPU/lib/python2.7/site-packages/ipykernel/__main__.py:267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/santanu/tensorflowGPU/lib/python2.7/site-packages/ipykernel/__main__.py:268: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/santanu/tensorflowGPU/lib/python2.7/site-packages/ipykernel/__main__.py:269: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.o

Checkpoint-4
[0]	train-mlogloss:1.08484	test-mlogloss:1.08497
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 20 rounds.
[10]	train-mlogloss:0.968686	test-mlogloss:0.970552
[20]	train-mlogloss:0.881821	test-mlogloss:0.885694
[30]	train-mlogloss:0.814872	test-mlogloss:0.820595
[40]	train-mlogloss:0.763279	test-mlogloss:0.770703
[50]	train-mlogloss:0.722458	test-mlogloss:0.731413
[60]	train-mlogloss:0.689608	test-mlogloss:0.700242
[70]	train-mlogloss:0.6632	test-mlogloss:0.675482
[80]	train-mlogloss:0.641167	test-mlogloss:0.655138
[90]	train-mlogloss:0.623203	test-mlogloss:0.638821
[100]	train-mlogloss:0.608049	test-mlogloss:0.625306
[110]	train-mlogloss:0.595006	test-mlogloss:0.613638
[120]	train-mlogloss:0.583837	test-mlogloss:0.603891
[130]	train-mlogloss:0.574177	test-mlogloss:0.595553
[140]	train-mlogloss:0.565577	test-mlogloss:0.588348
[150]	train-mlogloss:0.55826	test-mlogloss:0.582544
[160

In [22]:
train_X = sparse.hstack([train_df[features_to_use], tr_sparse])
test_X = sparse.hstack([test_df[features_to_use], te_sparse])

In [2]:
train_yy = []
for i in xrange(len(train_y)):
    if train_y[i] == 0:
        train_yy.append([1,0,0])
    if train_y[i] == 1:
        train_yy.append([0,1,0])
    if train_y[i] == 2:
        train_yy.append([0,0,1])

train_yy = np.array(train_yy)

,"num_rho","num_phi","num_cap_share","num_nr_of_lines","num_redacted","num_email","num_phone_nr"


NameError: name 'train_y' is not defined

In [144]:
tx = train_X.todense()
ts = test_X.todense()
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
train_yy= np_utils.to_categorical(train_y,3)
X1_train, X1_test, y1_train, y1_test = train_test_split(tx,train_yy, test_size=0.2, random_state=79)



from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD








print(X1_train.shape[0], 'train samples')
print(X1_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
#y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Dense(200, activation='relu', input_shape=(240,)))
model.add(Dropout(0.7))
model.add(BatchNormalization())
model.add(Dense(400, activation='relu',name='feature_layer'))
model.add(Dropout(0.7))
model.add(BatchNormalization())
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.7))
model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))

model.summary()
sgd = SGD(lr=1e-2, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])

callbacks = [
            EarlyStopping(monitor='val_loss', patience=20, verbose=0),
        ]

model.fit(X1_train, y1_train, batch_size=batch_size, nb_epoch=epochs,
              shuffle=True, verbose=2, validation_data=(X1_test,y1_test),
              callbacks=callbacks)

score = model.evaluate(X1_test, y1_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

39481 train samples
9871 test samples
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_35 (Dense)             (None, 200)               48200     
_________________________________________________________________
dropout_34 (Dropout)         (None, 200)               0         
_________________________________________________________________
batch_normalization_34 (Batc (None, 200)               800       
_________________________________________________________________
feature_layer (Dense)        (None, 400)               80400     
_________________________________________________________________
dropout_35 (Dropout)         (None, 400)               0         
_________________________________________________________________
batch_normalization_35 (Batc (None, 400)               1600      
_________________________________________________________________
dense_36 (Dense)             (None, 20

In [31]:
output = model.predict(ts)

In [32]:
output

array([[ 0.07686947,  0.22053415,  0.70259643],
       [ 0.07686947,  0.22053415,  0.70259643],
       [ 0.07686947,  0.22053415,  0.70259643],
       ..., 
       [ 0.07686947,  0.22053415,  0.70259643],
       [ 0.07686947,  0.22053415,  0.70259643],
       [ 0.07686947,  0.22053415,  0.70259643]], dtype=float32)

In [36]:
out_old = np.array(out_df)

In [45]:
out_df['high_1'] = output[:,0]
out_df['medium_1'] = output[:,0]
out_df['low_1'] = output[:,0]
out_df['high'] = 0.5*(out_df['high_1'] + out_df['high'])
out_df['medium'] = 0.5*(out_df['medium_1'] + out_df['medium'])
out_df['low'] = 0.5*(out_df['low'] + out_df['low'])
del out_df['low_1']
del out_df['medium_1']
del out_df['high_1']
out_df.to_csv('renthop_combined.csv',index=False)

In [46]:
out_df

,high,medium,low,listing_id
0,0.078858,0.102443,0.405260,7142618
1,0.073002,0.073393,0.963760,7210040
2,0.073086,0.083672,0.797944,7103890
3,0.075635,0.103750,0.435925,7143442
4,0.074189,0.086891,0.728808,6860601
5,0.072120,0.073927,0.969322,6840081
6,0.073142,0.085985,0.760051,6922337
7,0.077125,0.103793,0.411403,6913616
8,0.074059,0.095247,0.597191,6937820
9,0.074731,0.101113,0.492569,6893933


In [73]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint





49352
74659
Number of clusters: 107


In [88]:
out_df


,high,medium,low,listing_id
0,0.174651,0.677637,0.147711,7142618
1,0.020473,0.022587,0.956940,7210040
2,0.014333,0.273138,0.712530,7103890
3,0.151483,0.563975,0.284542,7143442
4,0.057541,0.274988,0.667471,6860601
5,0.000842,0.026639,0.972519,6840081
6,0.019735,0.230524,0.749741,6922337
7,0.081800,0.580989,0.337211,6913616
8,0.044385,0.364165,0.591450,6937820
9,0.050476,0.496974,0.452550,6893933


In [75]:
rep_points

,lat,lon
0,40.7514,-73.9730
1,40.7493,-73.7142
2,40.7601,-73.8221
3,41.0411,-73.5423
4,40.7863,-74.0112
5,40.7291,-74.2224
6,40.5956,-73.7555
7,39.8395,-86.1527
8,0.0000,0.0000
9,40.1159,-74.6267


In [59]:
test_df['display_address']

0        13274
1        13391
2          990
3          481
4        12317
5        14924
6        10056
7        12996
8        15294
9        12650
10        6813
11        9809
12        4111
13       12690
14       10876
15       10761
16        9652
17         180
18       11327
19        2760
20       15313
21       10107
22       10579
23         864
24       10761
25       14430
26       11676
27       14467
28       12823
29       15001
         ...  
74629      339
74630     8687
74631    10868
74632    15473
74633    11131
74634    10014
74635     9385
74636    13106
74637     9185
74638     3591
74639    11061
74640    12349
74641    15313
74642    10895
74643    10308
74644    14560
74645     9367
74646    15028
74647    10650
74648    11839
74649    13058
74650    15356
74651    14601
74652    15040
74653     8494
74654     7182
74655     8784
74656    15356
74657    11108
74658    12282
Name: display_address, dtype: int64

In [87]:
np.sum(cluster_labels)

10661

In [96]:
train_df['display_address']

0        12282
1         9080
2        13719
3        10866
4        15072
5        15194
6        14964
7        15223
8        11646
9         6459
10       15694
11       14685
12        9660
13       15380
14       10903
15       10791
16        2750
17       13340
18       14962
19        2752
20       12188
21        9584
22       10797
23       12060
24       13956
25       10901
26       10107
27       15369
28        1446
29       10066
         ...  
49322    12579
49323    14366
49324    11023
49325    12888
49326    15464
49327     8048
49328    13391
49329    14899
49330    10635
49331    15694
49332    15523
49333    15371
49334    12499
49335     8636
49336    10839
49337    10028
49338    11088
49339    10989
49340    11228
49341    11841
49342    14310
49343    10020
49344     8953
49345    11106
49346    10579
49347     9577
49348    12864
49349    15306
49350    14850
49351    12707
Name: display_address, dtype: int64

In [91]:
train_df['display_address']

0        12282
1         9080
2        13719
3        10866
4        15072
5        15194
6        14964
7        15223
8        11646
9         6459
10       15694
11       14685
12        9660
13       15380
14       10903
15       10791
16        2750
17       13340
18       14962
19        2752
20       12188
21        9584
22       10797
23       12060
24       13956
25       10901
26       10107
27       15369
28        1446
29       10066
         ...  
49322    12579
49323    14366
49324    11023
49325    12888
49326    15464
49327     8048
49328    13391
49329    14899
49330    10635
49331    15694
49332    15523
49333    15371
49334    12499
49335     8636
49336    10839
49337    10028
49338    11088
49339    10989
49340    11228
49341    11841
49342    14310
49343    10020
49344     8953
49345    11106
49346    10579
49347     9577
49348    12864
49349    15306
49350    14850
49351    12707
Name: display_address, dtype: int64

In [106]:
train_abc = pd.read_json("~/Downloads/train.json")

In [109]:
train_abc['levestein'] = train_abc.apply(lambda row: graphlab.distances.levenshtein(row['street_address'][0],row['display_address'][0]),axis=1)

IndexError: ('string index out of range', u'occurred at index 100544')

In [100]:
import graphlab 

graphlab.distances.levenshtein("i am a good boy", "i am a very good boy")
graphlab.distances.cosine("i am a good boy", "i am a very good boy")

ToolkitError: This distance does not support the provided type.

In [1]:
train_abc = pd.read_json("~/Downloads/train.json")[['street_address','display_address']]
test_abc  = pd.read_json("~/Downloads/test.json")[['street_address','display_address']]
tt_abc = train_abc.append(test_abc)
levestein_distance = []
cosine_distance  = []
street_address = np.array(tt_abc['street_address'])
display_address = np.array(tt_abc['display_address'])
for i in xrange(len(tt_abc)):
    levestein_distance.append(graphlab.distances.levenshtein(street_address[i].lower(),display_address[i].lower()))
    



NameError: name 'pd' is not defined

In [125]:
levestein_distance

[4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 3.0,
 0.0,
 5.0,
 4.0,
 4.0,
 3.0,
 4.0,
 4.0,
 4.0,
 3.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 3.0,
 3.0,
 4.0,
 4.0,
 4.0,
 0.0,
 4.0,
 0.0,
 3.0,
 4.0,
 3.0,
 4.0,
 4.0,
 8.0,
 3.0,
 2.0,
 4.0,
 3.0,
 4.0,
 4.0,
 5.0,
 4.0,
 3.0,
 4.0,
 4.0,
 5.0,
 15.0,
 6.0,
 0.0,
 3.0,
 16.0,
 4.0,
 6.0,
 4.0,
 4.0,
 0.0,
 4.0,
 6.0,
 4.0,
 4.0,
 4.0,
 4.0,
 5.0,
 4.0,
 4.0,
 4.0,
 4.0,
 0.0,
 4.0,
 3.0,
 4.0,
 3.0,
 4.0,
 4.0,
 3.0,
 6.0,
 5.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 3.0,
 4.0,
 4.0,
 4.0,
 4.0,
 3.0,
 4.0,
 4.0,
 3.0,
 7.0,
 2.0,
 4.0,
 5.0,
 4.0,
 0.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 5.0,
 2.0,
 4.0,
 4.0,
 5.0,
 4.0,
 4.0,
 0.0,
 6.0,
 11.0,
 4.0,
 3.0,
 4.0,
 4.0,
 4.0,
 4.0,
 0.0,
 4.0,
 4.0,
 7.0,
 4.0,
 0.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 21.0,
 4.0,
 3.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 5.0,
 3.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 3.0,
 4.0,
 2.0,
 4.0,
 0.0,
 0.0,
 4.0,
 4.0,
 3.0,
 0.0,
 4.0,
 4.0,


/home/santanu/tensorflowGPU/lib/python2.7/site-packages/ipykernel/__main__.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/santanu/tensorflowGPU/lib/python2.7/site-packages/ipykernel/__main__.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/santanu/tensorflowGPU/lib/python2.7/site-packages/ipykernel/__main__.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [ ]:
location_dict = {
'manhatten_loc' : [40.7527, -73.9943],
'brooklyn_loc' : [45.0761,-73.9442],
'bronx_loc' : [40.8448,-73.8648],
'queens_loc' : [40.7282,-73.7949],
'staten_loc' : [40.5795,-74.1502]}

for location in location_dict.keys():

    lat1 = train_df['latitude'].apply(radians)
    lon1 = train_df['longitude'].apply(radians)
    lat2 = radians(location_dict[location][0])
    lon2 = radians(location_dict[location][1])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    def power(x):
        return x**2

    a = (dlat/2).apply(sin).apply(power) + lat1.apply(cos) * cos(lat2) * (dlon/2).apply(sin).apply(power)
    c = 2 * a.apply(sqrt).apply(sin)

    ### Add a new column called distance
    train_df['distance_' + location] = R * c
    features_to_use.append('distance_' + location)

In [134]:
def runXGB(train_X, train_y, test_X,test_y,test_final,feature_names=None, seed_val=321, num_rounds=2000):
    param = {}
    param['objective'] = 'multi:softprob'
    param['eta'] = 0.02
    param['max_depth'] = 6
    param['silent'] = 0
    param['num_class'] = 3
    param['eval_metric'] = "mlogloss"
    param['min_child_weight'] = 20
    #param['subsample'] = 0.7
    #param['colsample_bytree'] = 0.7
    param['lambda'] = 2
    param['alpha'] = 0.02
    param['subsample'] = 0.8
    param['colsample_bytree'] = 0.8
    param['seed'] = seed_val
    num_rounds = num_rounds

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist,verbose_eval=10,early_stopping_rounds=20)
    
    xgtest1 = xgb.DMatrix(test_final)
    pred_test_y = model.predict(xgtest1)
    return pred_test_y, model

preds, model = runXGB(X_train,y_train,X_test,y_test,test_X, num_rounds=2000)
out_df = pd.DataFrame(preds)
out_df.columns = ["high", "medium", "low"]
out_df["listing_id"] = test_df.listing_id.values
out_df.to_csv("sub54_new.csv", index=False)

[0]	train-mlogloss:1.08467	test-mlogloss:1.08482
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 20 rounds.
[10]	train-mlogloss:0.972106	test-mlogloss:0.973233
[20]	train-mlogloss:0.886912	test-mlogloss:0.889152
[30]	train-mlogloss:0.821269	test-mlogloss:0.824471
[40]	train-mlogloss:0.7719	test-mlogloss:0.776105
[50]	train-mlogloss:0.7323	test-mlogloss:0.737616
[60]	train-mlogloss:0.701934	test-mlogloss:0.708262
[70]	train-mlogloss:0.677584	test-mlogloss:0.68496
[80]	train-mlogloss:0.657362	test-mlogloss:0.665795
[90]	train-mlogloss:0.639537	test-mlogloss:0.64918
[100]	train-mlogloss:0.624053	test-mlogloss:0.634789
[110]	train-mlogloss:0.610652	test-mlogloss:0.622472
[120]	train-mlogloss:0.598759	test-mlogloss:0.611913
[130]	train-mlogloss:0.588451	test-mlogloss:0.602725
[140]	train-mlogloss:0.579688	test-mlogloss:0.595097
[150]	train-mlogloss:0.572618	test-mlogloss:0.589001
[160]	train-mlogloss

In [ ]:
from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from keras.layers.normalization import BatchNormalization



batch_size = 1000
epochs = 20

# the data, shuffled and split between train and test sets
X_train,y_train,X_test,y_test




X_train = X_train.astype('float32').todense()
X_test = X_test.astype('float32').todense()

print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
#y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Dense(300, activation='relu', input_shape=(240,)))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(300, activation='relu',name='feature_layer'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
#model.add(Dense(300, activation='relu'))
#model.add(Dropout(0.2))
#model.add(BatchNormalization())
model.add(Dense(1, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(X_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

<39481x240 sparse matrix of type '<type 'numpy.float64'>'
	with 1627275 stored elements in Compressed Sparse Row format>

In [139]:
np.shape(X_train)

(39481, 240)

In [ ]:
Xg = graphlab.SFrame(X_train.todense())

In [2]:
print('abc')

abc


60